In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Data from 2011 - 2014

In [ ]:
%pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import chardet
file_path = 'Superstore.csv'
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
    return result['encoding']

detected_encoding = detect_encoding(file_path)
print(f"Detected encoding: {detected_encoding}")


Detected encoding: Windows-1252


In [ ]:
ss = pd.read_csv('Superstore.csv', encoding=detected_encoding)
ss.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2013-152156,09-11-2013,12-11-2013,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2013-152156,09-11-2013,12-11-2013,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2013-138688,13-06-2013,17-06-2013,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2012-108966,11-10-2012,18-10-2012,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2012-108966,11-10-2012,18-10-2012,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
ss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

# Pricing strategy

Sales by Category

In [ ]:
ss.groupby('Category')['Sales'].sum().sort_values(ascending=False)

Category
Technology         836154.0330
Furniture          741999.7953
Office Supplies    719047.0320
Name: Sales, dtype: float64

Technology has the most sales among all categories.

In [ ]:
ss[ss['Category'] == 'Technology'].groupby('Sub-Category')['Sales'].sum().sort_values(ascending=False)

Sub-Category
Phones         330007.054
Machines       189238.631
Accessories    167380.318
Copiers        149528.030
Name: Sales, dtype: float64

Phones has the most sales in technology category.

In [ ]:
ss[ss['Sub-Category'] == 'Phones'].groupby('Product Name')['Sales'].sum().sort_values(ascending=False)

Product Name
Samsung Galaxy Mega 6.3                                                13943.668
Apple iPhone 5                                                         12996.600
Wilson Electronics DB Pro Signal Booster                                8878.400
Mitel MiVoice 5330e IP Phone                                            7699.720
Samsung Galaxy S III - 16GB - pebble blue (T-Mobile)                    7139.796
                                                                         ...    
Shocksock Galaxy S4 Armband                                               63.510
Anker 24W Portable Micro USB Car Charger                                  61.544
SKILCRAFT Telephone Shoulder Rest, 2" x 6.5" x 2.5", Black                56.164
Sannysis Cute Owl Design Soft Skin Case Cover for Samsung Galaxy S4       13.860
PNY Rapid USB Car Charger - Black                                          9.588
Name: Sales, Length: 189, dtype: float64

Samsung Galaxy Mega 6.3 and Apple iphone 5 have the most sales in phone sub-category.

Profit by category

In [ ]:
ss.groupby('Category')['Profit'].sum().sort_values(ascending=False)

Category
Technology         145454.9481
Office Supplies    122490.8008
Furniture           18451.2728
Name: Profit, dtype: float64

In [ ]:
ss[ss['Category'] == 'Technology']['Profit'].std()/ss[ss['Category'] == 'Office Supplies']['Profit'].std()

2.6006653372152964

In [ ]:
import scipy.stats as stats
data1 = ss[ss['Category'] == 'Technology']['Profit']
data2 = ss[ss['Category'] == 'Office Supplies']['Profit']
t_statistic, p_value = stats.ttest_ind(data1, data2)

# Print the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Check if the p-value is significant (using a common significance level of 0.05)
if p_value < 0.05:
    print("The means are significantly different.")
else:
    print("The means are not significantly different.")

t-statistic: 8.687539082544111
p-value: 4.4542772738946324e-18
The means are significantly different.


In [ ]:
print('tech avg : {}'.format(data1.mean()))
print('office supplies avg : {}'.format(data2.mean()))

tech avg : 78.75200221981586
office supplies avg : 20.327049585131117


Technology has the most profit among all categories.

In [ ]:
ss[ss['Category'] == 'Technology'].groupby('Sub-Category')['Sales'].sum().sort_values(ascending=False)

Sub-Category
Phones         330007.054
Machines       189238.631
Accessories    167380.318
Copiers        149528.030
Name: Sales, dtype: float64

In [ ]:
ss[ss['Sub-Category'] == 'Phones'].groupby('Product Name')['Profit'].sum().sort_values(ascending=False)

Product Name
Samsung Galaxy Mega 6.3                                                                                 1822.7566
Apple iPhone 5                                                                                          1611.5784
Panasonic KX-TG9471B                                                                                    1168.1004
Panasonic KX-TG9541B DECT 6.0 Digital 2-Line Expandable Cordless Phone With Digital Answering System    1147.4235
Samsung Galaxy S4 Mini                                                                                  1104.4765
                                                                                                          ...    
Cisco IP Phone 7961G-GE VoIP phone                                                                      -119.1190
Samsung Galaxy Note 2                                                                                   -128.7972
LG Exalt                                                                   

If we have a pricing strategy, we should focus on upselling in the phone-technology category, particularly for the Samsung Galaxy Mega 6.3 and the Apple iPhone 5

In [ ]:
ss.groupby('Product Name')['Order ID'].nunique().sort_values(ascending=False)/ss['Order ID'].nunique()*100

Product Name
Staples                                                                      4.432022
Avery Non-Stick Binders                                                      0.399281
KI Adjustable-Height Table                                                   0.359353
Storex Dura Pro Binders                                                      0.339389
Logitech 910-002974 M325 Wireless Mouse for Web Scrolling                    0.299461
                                                                               ...   
Bush Birmingham Collection Bookcase, Dark Cherry                             0.019964
Sony 8GB Class 10 Micro SDHC R40 Memory Card                                 0.019964
Socket Bluetooth Cordless Hand Scanner (CHS)                                 0.019964
Snap-A-Way Black Print Carbonless Speed Message, No Reply Area, Duplicate    0.019964
Cisco 8961 IP Phone Charcoal                                                 0.019964
Name: Order ID, Length: 1841, dtype: floa

The most commonly purchased product in every basket is staples, accounting for 4.43% of all baskets. It seems less in value. Lets, deep down into month.

In [ ]:
ss['Order Date'] = pd.to_datetime(ss['Order Date'])
ss['Ship Date'] = pd.to_datetime(ss['Ship Date'])

In [ ]:
import datetime as dt
ss['Order Month'] = ss['Order Date'].dt.month

In [ ]:
a = ss.groupby(['Order Month','Product Name'])['Order ID'].nunique()/ss.groupby('Order Month')['Order ID'].nunique()*100
a = pd.DataFrame(a)
a.columns = ['Percent']
for i in range(1,13):
    print('Month {}'.format(i))
    print(a.loc[i].sort_values(by=['Percent'],ascending=False).head())
    print('*******************************')

Month 1
                                                     Percent
Product Name                                                
Staples                                             3.956835
Wilson Jones data.warehouse D-Ring Binders with...  1.079137
Eldon ClusterMat Chair Mat with Cordless Antist...  1.079137
DAX Black Cherry Wood-Tone Poster Frame             1.079137
Global Highback Leather Tilter in Burgundy          1.079137
*******************************
Month 2
                                                     Percent
Product Name                                                
Staples                                             8.661417
Adjustable Depth Letter/Legal Cart                  1.181102
Honeywell Enviracaire Portable HEPA Air Cleaner...  1.181102
GBC Plastic Binding Combs                           1.181102
Wilson Jones Easy Flow II Sheet Lifters             1.181102
*******************************
Month 3
                                                     Perce

Seem less amount.

Lets do some .Create matrix 0,1 on order id vs product name as input of association rule.

In [ ]:
!pip install mlxtend

In [ ]:
df = ss[['Order ID','Product Name']]
encoded_df = pd.get_dummies(df, columns=['Product Name'], prefix='', prefix_sep='')
encoded_df

,Order ID,"""While you Were Out"" Message Book, One Form per Page","#10 Gummed Flap White Envelopes, 100/Box",#10 Self-Seal White Envelopes,"#10 White Business Envelopes,4 1/8 x 9 1/2","#10- 4 1/8"" x 9 1/2"" Recycled Envelopes","#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes","#10-4 1/8"" x 9 1/2"" Premium Diagonal Seam Envelopes",#6 3/4 Gummed Flap White Envelopes,"1.7 Cubic Foot Compact ""Cube"" Office Refrigerators",...,Zebra ZM400 Thermal Label Printer,Zebra Zazzle Fluorescent Highlighters,Zipper Ring Binder Pockets,i.Sound Portable Power - 8000 mAh,iHome FM Clock Radio with Lightning Dock,"iKross Bluetooth Portable Keyboard + Cell Phone Stand Holder + Brush for Apple iPhone 5S 5C 5, 4S 4",iOttie HLCRIO102 Car Mount,iOttie XL Car Mount,invisibleSHIELD by ZAGG Smudge-Free Screen Protector,netTALK DUO VoIP Telephone Service
0,CA-2013-152156,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CA-2013-152156,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CA-2013-138688,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,US-2012-108966,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,US-2012-108966,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,CA-2011-110422,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9990,CA-2014-121258,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9991,CA-2014-121258,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9992,CA-2014-121258,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Generate frequent itemsets using Apriori algorithm
frequent_itemsets = apriori(encoded_df.drop('Order ID', axis=1), min_support=0.05, use_colnames=True)

# Generate association rules
association_rules_df = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Display association rules
print("Frequent Itemsets:")
print(frequent_itemsets)
print("\nAssociation Rules:")
print(association_rules_df)

F:\Python\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The input DataFrame `df` containing the frequent itemsets is empty.

There is no association rule beacause all product didn't pass minimum support and minimum threshold.

# Supply Chain aspect

find out time to ship order. We assume that order date is the day that order was made and ship date is the day when order was delivered.

In [ ]:
ss['delivery time'] = ss['Ship Date'] - ss['Order Date']
ss['delivery time'].describe()

count                       9994
mean      8 days 00:41:12.523514
std      96 days 04:26:57.728617
min          -320 days +00:00:00
25%              2 days 00:00:00
50%              4 days 00:00:00
75%             61 days 00:00:00
max            214 days 00:00:00
Name: delivery time, dtype: object

It seems that min delivery time has negative value. So, we assume that it was mistakes in recording value. We solve this by absoluting their values.

In [ ]:
from datetime import timedelta
ss[ss['delivery time'] == timedelta(days=-320)]

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Order Month,delivery time
716,717,CA-2011-130092,2011-12-01,2011-01-15,First Class,SV-20365,Seth Vernon,Consumer,United States,Dover,...,FUR-FU-10000010,Furniture,Furnishings,"DAX Value U-Channel Document Frames, Easel Back",9.940,2,0.0,3.0814,12,-320 days
961,962,CA-2012-160059,2012-11-27,2012-01-12,Standard Class,TB-21190,Thomas Brumley,Home Office,United States,Fayetteville,...,OFF-BI-10000145,Office Supplies,Binders,Zipper Ring Binder Pockets,6.240,2,0.0,3.0576,11,-320 days
2211,2212,CA-2013-162313,2013-11-28,2013-01-12,First Class,VB-21745,Victoria Brennan,Corporate,United States,Lincoln Park,...,OFF-AP-10003842,Office Supplies,Appliances,Euro-Pro Shark Turbo Vacuum,167.292,6,0.1,29.7408,11,-320 days
3475,3476,CA-2013-144911,2013-11-28,2013-01-12,First Class,RW-19630,Rob Williams,Corporate,United States,Overland Park,...,TEC-AC-10004633,Technology,Accessories,Verbatim 25 GB 6x Blu-ray Single Layer Recorda...,34.950,5,0.0,15.3780,11,-320 days
3476,3477,CA-2013-144911,2013-11-28,2013-01-12,First Class,RW-19630,Rob Williams,Corporate,United States,Overland Park,...,OFF-BI-10000977,Office Supplies,Binders,Ibico Plastic Spiral Binding Combs,152.000,5,0.0,69.9200,11,-320 days
3793,3794,CA-2012-163762,2012-11-27,2012-01-12,Standard Class,JE-15475,Jeremy Ellison,Consumer,United States,New York City,...,OFF-AR-10001915,Office Supplies,Art,Peel-Off China Markers,29.790,3,0.0,12.5118,11,-320 days
4337,4338,CA-2012-125234,2012-11-27,2012-01-12,Standard Class,SN-20710,Steve Nguyen,Home Office,United States,Los Angeles,...,TEC-AC-10000158,Technology,Accessories,Sony 64GB Class 10 Micro SDHC R40 Memory Card,107.970,3,0.0,22.6737,11,-320 days
4338,4339,CA-2012-125234,2012-11-27,2012-01-12,Standard Class,SN-20710,Steve Nguyen,Home Office,United States,Los Angeles,...,OFF-PA-10000482,Office Supplies,Paper,Snap-A-Way Black Print Carbonless Ruled Speed ...,113.820,3,0.0,53.4954,11,-320 days
5192,5193,CA-2011-101560,2011-11-28,2011-01-12,Second Class,CS-12250,Chris Selesnick,Corporate,United States,Columbia,...,FUR-FU-10003773,Furniture,Furnishings,Eldon Cleatmat Plus Chair Mats for High Pile C...,397.600,5,0.0,43.7360,11,-320 days
5193,5194,CA-2011-101560,2011-11-28,2011-01-12,Second Class,CS-12250,Chris Selesnick,Corporate,United States,Columbia,...,OFF-PA-10001363,Office Supplies,Paper,Xerox 1933,85.960,7,0.0,40.4012,11,-320 days
